In [1]:
import numpy as np

# 工序的加工时间
op_pt = np.array([[48, 74, 33],
                  [47, 97, 0],
                  [84, 79, 37],
                  [97, 81, 69],
                  [0, 0, 68],
                  [3, 85, 40],
                  [67, 85, 48],
                  [0, 49, 8],
                  [93, 53, 98],
                  [86, 0, 28],
                  [35, 98, 77],
                  [41, 0, 70],
                  [65, 0, 0],
                  [0, 11, 23],
                  [78, 19, 16]])

# 机器的加工能耗
mch_working_power = np.array([0.88138383, 0.68393263, 0.93675113])

# 将无法加工的工序设置为无穷大能耗
op_energy = np.where(op_pt == 0, np.inf, op_pt * mch_working_power)

# 对于每个工序，找到能耗最低的机器，并计算该工序的最低能耗
min_energy_per_op = np.min(op_energy, axis=1)

# 计算总的能耗下限
total_min_energy = np.sum(min_energy_per_op)

total_min_energy


488.46801371000004

In [3]:
# 机器待机能耗
mch_idle_power = np.array([0.1884805 , 0.14602384, 0.11931703])

# 计算每个工序在每台机器上的加工能耗，如果不能加工则为无穷大
op_energy = np.where(op_pt == 0, np.inf, op_pt * mch_working_power)

# 对于每个工序，找到最低能耗的机器
min_energy_per_op = np.min(op_energy, axis=1)

# 找到每个工序最低能耗对应的机器索引
min_energy_mch_index = np.argmin(op_energy, axis=1)

# 初始化机器的总工作时间
total_mch_work_time = np.zeros_like(mch_working_power)

# 对于每个工序，累加对应机器的工作时间
for op_index, mch_index in enumerate(min_energy_mch_index):
    if op_pt[op_index, mch_index] != 0:  # 确保工序可以在选定的机器上加工
        total_mch_work_time[mch_index] += op_pt[op_index, mch_index]

# 计算机器工作期间的总能耗
total_work_energy = np.sum(min_energy_per_op)

# 计算机器待机期间的总能耗，这里简化为0，因为没有考虑具体的待机时间
# 实际上，如果想考虑待机能耗，需要根据实际生产计划来估计
# total_idle_energy = np.sum(mch_idle_power * total_mch_idle_time)  # 这里我们没有total_mch_idle_time的具体值

# 总能耗为工作能耗加上待机能耗
total_energy = total_work_energy # + total_idle_energy

total_energy, total_mch_work_time


(488.46801371000004, array([191., 164., 222.]))

In [ ]:
# 找到完成所有工序所需的最长时间，即所有机器的最大工作时间
max_work_time = np.max(total_mch_work_time)

# 对于每台机器，计算其待机时间
idle_time_per_machine = max_work_time - total_mch_work_time

# 计算待机期间的能耗
idle_energy_per_machine = idle_time_per_machine * mch_idle_power

# 计算总的待机能耗
total_idle_energy = np.sum(idle_energy_per_machine)

# 更新总能耗，包括待机能耗
total_energy_including_idle = total_work_energy + total_idle_energy

total_energy_including_idle, total_idle_energy


In [4]:

# 工件数和每个工件的工序数
job_length_list = np.array([3, 3, 3, 3, 3])

# 计算每个工序在每台机器上的加工能耗，如果不能加工则为无穷大
op_energy_max = np.where(op_pt == 0, np.inf, op_pt * mch_working_power)

# 对于每个工序，找到能耗最高的机器
max_energy_per_op = np.max(np.where(op_pt > 0, op_energy_max, 0), axis=1)

# 计算工作能耗的最大值
total_max_work_energy = np.sum(max_energy_per_op)

# 为了简化，假设每个工件的工序之间的最大等待时间为该工件所有工序的最大加工时间的总和
# 这是一个非常保守的估计，实际的待机时间可能会少很多
# 找到每个工件所有工序的最大加工时间

# 重新计算每个工件所有工序的最大加工时间
max_op_time_per_job = np.array([np.max(op_pt[i*3:(i+1)*3], axis=0) for i in range(len(job_length_list))])

# 估计每个工件的最大等待时间
max_wait_time_per_job = np.sum(max_op_time_per_job, axis=1)

# 假设在最坏情况下，所有机器在等待期间都处于待机状态
# 使用最大加工时间的总和乘以最大的机器待机功率来估计最大待机能耗
total_max_idle_energy = np.sum(max_wait_time_per_job) * np.max(mch_idle_power)

# 更新总能耗的最大估计值，包括待机能耗
total_energy_max_estimate = total_max_work_energy + total_max_idle_energy

total_energy_max_estimate


1155.99696174

In [ ]:
# 重新导入numpy库并重新定义之前的变量和计算
import numpy as np

# 定义变量
op_pt = np.array([[48, 74, 33],
                  [47, 97, 0],
                  [84, 79, 37],
                  [97, 81, 69],
                  [0, 0, 68],
                  [3, 85, 40],
                  [67, 85, 48],
                  [0, 49, 8],
                  [93, 53, 98],
                  [86, 0, 28],
                  [35, 98, 77],
                  [41, 0, 70],
                  [65, 0, 0],
                  [0, 11, 23],
                  [78, 19, 16]])
mch_working_power = np.array([0.88138383, 0.68393263, 0.93675113])

# 计算每个工序的最长加工时间和对应的机器索引
max_time_per_op = np.max(op_pt, axis=1)  # 每个工序的最长加工时间
max_time_mch_index = np.argmax(op_pt, axis=1)  # 最长加工时间对应的机器索引

# 初始化每台机器的工作时间为0
machine_times = np.zeros(len(mch_working_power))

# 对于每个工件，计算其完成时间
job_length_list = np.array([3, 3, 3, 3, 3])
job_completion_times = []
for job_idx in range(len(job_length_list)):
    job_time = 0
    for op_idx in range(job_length_list[job_idx]):
        global_op_idx = job_idx * 3 + op_idx
        mch_idx = max_time_mch_index[global_op_idx]
        op_time = max_time_per_op[global_op_idx]
        
        machine_available_time = machine_times[mch_idx]
        start_time = max(machine_available_time, job_time)
        machine_times[mch_idx] = start_time + op_time
        job_time = start_time + op_time
    
    job_completion_times.append(job_time)

completion_time_upper_bound = max(job_completion_times)
completion_time_upper_bound
